# CoppeliaSim: PART 2
* Run the scene workshop1.ttt in CoppeliaSim

In [4]:
import sim
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Check if python is connecting to Coppelia

In [5]:
sim.simxFinish(-1)  # Close opened connections
clientID = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim

if clientID != -1:
    print('Connected')

    # Now try to retrieve data in a blocking fashion (i.e. a service call):
    res, objs = sim.simxGetObjects(clientID, sim.sim_handle_all, sim.simx_opmode_blocking)

    print('Simulation time in milliseconds: ', sim.simxGetLastCmdTime(clientID))
    
    # Get Object position
    name = 'Omnirob'
    err_code, cuboid = sim.simxGetObjectHandle(clientID, name, sim.simx_opmode_blocking)
    res, position = sim.simxGetObjectPosition(clientID, cuboid, -1, sim.simx_opmode_blocking)        
    print('Omnirob is at [x,y,z]=', position)
    
    # Now close the connection to CoppeliaSim:
    sim.simxGetPingTime(clientID)
    sim.simxFinish(clientID)
    print('Disconnected')
else:
    print('Failed connecting to remote API server')


Connected
Simulation time in milliseconds:  11250
Omnirob is at [x,y,z]= [-2.9997830390930176, -0.07474494725465775, 0.3724490702152252]
Disconnected


# Robot

In [6]:
class robot():
    
    def __init__(self, frame_name, motor_names=[], client_id=0):  
        # If there is an existing connection
        if client_id:
                self.client_id = client_id
        else:
            self.client_id = self.open_connection()
            
        self.motors = self._get_handlers(motor_names) 
        
        # Robot frame
        self.frame =  self._get_handler(frame_name)
            
        
    def open_connection(self):
        sim.simxFinish(-1)  # just in case, close all opened connections
        self.client_id = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim 
        
        if clientID != -1:
            print('Robot connected')
        else:
            print('Connection failed')
        return clientID
        
    def close_connection(self):    
        sim.simxGetPingTime(self.client_id)  # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive.
        sim.simxFinish(self.client_id)  # Now close the connection to CoppeliaSim:
        print('Connection closed')
    
    def isConnected(self):
        c,result = sim.simxGetPingTime(self.client_id)
        # Return true if the robot is connected
        return result > 0         
        
    def _get_handler(self, name):
        err_code, handler = sim.simxGetObjectHandle(self.client_id, name, sim.simx_opmode_blocking)
        return handler
    
    def _get_handlers(self, names):
        handlers = []
        for name in names:
            handler = self._get_handler(name)
            handlers.append(handler)
        
        return handlers

    def send_motor_velocities(self, vels):
        for motor, vel in zip(self.motors, vels):
            err_code = sim.simxSetJointTargetVelocity(self.client_id, 
                                                      motor, vel, sim.simx_opmode_streaming)      
            
    def set_position(self, position, relative_object=-1):
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)        
        sim.simxSetObjectPosition(clientID, self.frame, relative_object, position, sim.simx_opmode_oneshot)
        
    def simtime(self):
        return sim.simxGetLastCmdTime(self.client_id)
    
    def get_position(self, relative_object=-1):
        # Get position relative to an object, -1 for global frame
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)
        res, position = sim.simxGetObjectPosition(self.client_id, self.frame, relative_object, sim.simx_opmode_blocking)        
        return array(position)
    
    def get_object_position(self, object_name):
        # Get Object position in the world frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, -1, sim.simx_opmode_blocking)
        return array(position)
    
    def get_object_relative_position(self, object_name):        
        # Get Object position in the robot frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, self.frame, sim.simx_opmode_blocking)
        return array(position)

In [97]:
# Joint names in coppelia for KUKA omnidirectional robot
motor_names = ['Omnirob_FLwheel_motor', 'Omnirob_FRwheel_motor', 'Omnirob_RRwheel_motor', 'Omnirob_RLwheel_motor']


r = robot('Omnirob', motor_names)  # Create an instance of our robot

#left
r.send_motor_velocities([-1, -1, 1, 1])

#right
r.send_motor_velocities([1, 1, -1, -1])

time.sleep(1)

r.close_connection()  # End

Robot connected
Connection closed


In [ ]:
# Joint names in coppelia for KUKA omnidirectional robot
motor_names = ['Omnirob_FLwheel_motor', 'Omnirob_FRwheel_motor', 'Omnirob_RRwheel_motor', 'Omnirob_RLwheel_motor']


r = robot('Omnirob', motor_names)  # Create an instance of our robot

right = r._get_handler('sensor_right')
left = r._get_handler('sensor_left')
mid = r._get_handler('sensor_mid')
print(right)
print(left)
vx = 0
vy = -3
iteration = 0
direction = 0
while True:
    res, res2, m_reading = sim.simxReadVisionSensor(r.client_id, mid, sim.simx_opmode_streaming)
    if m_reading:
        print(m_reading[0][6])
        if m_reading[0][6] > 0.95:
            vx = 0
            vy = 0
            time.sleep(0.6)
            r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])
            r.close_connection()  # End
            break
    
    res, res2, r_reading = sim.simxReadVisionSensor(r.client_id, right, sim.simx_opmode_streaming)
    if r_reading:
        if r_reading[0][11] < 0.2:
            #turn right
            if direction == right:
                vx = vx + -0.005
                vy = -3
            else:
                vx = -1
                vy = -3
                direction = right
        res, res2, L_reading = sim.simxReadVisionSensor(r.client_id, left, sim.simx_opmode_streaming)
        if L_reading:
            if L_reading[0][11] < 0.2:
                #turn left
                if direction == left:
                    vx = vx + 0.005
                    vy = -3
                else:
                    vx = +1
                    vy = -3
                    direction = left
            else:
                pass
    r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])
r.close_connection()  # End

Robot connected
51
50
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509

0.8313725590705872
0.8313725590705872
0.8313725590705872
0.8313725590705872
0.8313725590705872
0.8313725590705872
0.8313725590705872
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274

0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8352941274642944
0.8392156958580017
0.8392156958580017
0.8392156958580017
0.8392156958580017
0.8392156958580017
0.8392156958580017
0.8392156958580017
0.8392156958580017
0.8392156958580017
0.8392156958580017
0.8392156958580017
0.8392156958580017
0.8392156958580017
0.8392156958580017
0.8392156958

0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8745098114013672
0.8627451062202454
0.8627451062202454
0.8627451062202454
0.8627451062202454
0.8627451062202454
0.8627451062202454
0.8627451062202454
0.8627451062202454
0.8627451062202454
0.8627451062202454
0.8627451062202454
0.8627451062202454
0.8627451062202454
0.8627451062202454
0.8627451062202454
0.8627451062202454
0.8627451062202454
0.8627451062202454
0.8627451062202454
0.8627451062

0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.09019608050584793
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.8274509906768799
0.827

...

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
